In [ ]:
import pandas as pd
datos = pd.read_csv("clientes.csv")

In [55]:
datos.head()

,CompanyName,SalesPerson,Phone,PasswordHash,PasswordSalt,ModifiedDate
0,A Bike Store,adventure-works\pamela0,245-555-0173,L/Rlwxzp4w7RWmEgXX+/A7cXaePEPcp+KwQhl2fJL7w=,1KjXYs4=,2005-08-01 00:00:00.000
1,Progressive Sports,adventure-works\david8,170-555-0127,YPdtRdvqeAhj6wyxEsFdshBDNXxkCXn+CRgbvJItknw=,fs1ZGhY=,2006-08-01 00:00:00.000
2,Advanced Bike Components,adventure-works\jillian0,279-555-0130,LNoK27abGQo48gGue3EBV/UrlYSToV0/s87dCRV7uJk=,YTNH5Rw=,2005-09-01 00:00:00.000
3,Modular Cycle Systems,adventure-works\jillian0,710-555-0173,ElzTpSNbUW1Ut+L5cWlfR7MF6nBZia8WpmGaQPjLOJA=,nm7D5e4=,2006-07-01 00:00:00.000
4,Metropolitan Sports Supply,adventure-works\shu0,828-555-0186,KJqV15wsX3PG8TS5GSddp6LFFVdd3CoRftZM/tP0+R4=,cNFKU4w=,2006-09-01 00:00:00.000


In [56]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
import re
# def sigmoid(x):
#     return 1 / (1 + np.exp(-x))



datos['Phone'] = datos['Phone'].apply(lambda x: int(re.sub(r'\D', '', x)[-10:-7]))
datos['ModifiedDate'] = pd.to_datetime(datos['ModifiedDate'])
datos['ModifiedDate'] = datos['ModifiedDate'].apply(lambda x: int(x.timestamp()))

# One hot enconde company name and sales person
datos = pd.get_dummies(datos, columns=['CompanyName', 'SalesPerson'], drop_first=True)


In [58]:
datos.head()

,Phone,PasswordHash,PasswordSalt,ModifiedDate,CompanyName_A Cycle Shop,CompanyName_A Great Bicycle Company,CompanyName_A Typical Bike Shop,CompanyName_Acceptable Sales & Service,CompanyName_Action Bicycle Specialists,CompanyName_Active Life Toys,...,CompanyName_Year-Round Sports,CompanyName_Yellow Bicycle Company,SalesPerson_adventure-works\garrett1,SalesPerson_adventure-works\jae0,SalesPerson_adventure-works\jillian0,SalesPerson_adventure-works\josé1,SalesPerson_adventure-works\linda3,SalesPerson_adventure-works\michael9,SalesPerson_adventure-works\pamela0,SalesPerson_adventure-works\shu0
0,245,L/Rlwxzp4w7RWmEgXX+/A7cXaePEPcp+KwQhl2fJL7w=,1KjXYs4=,1122854400,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,170,YPdtRdvqeAhj6wyxEsFdshBDNXxkCXn+CRgbvJItknw=,fs1ZGhY=,1154390400,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,279,LNoK27abGQo48gGue3EBV/UrlYSToV0/s87dCRV7uJk=,YTNH5Rw=,1125532800,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,710,ElzTpSNbUW1Ut+L5cWlfR7MF6nBZia8WpmGaQPjLOJA=,nm7D5e4=,1151712000,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
4,828,KJqV15wsX3PG8TS5GSddp6LFFVdd3CoRftZM/tP0+R4=,cNFKU4w=,1157068800,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [61]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score


X = datos.drop(columns=['ModifiedDate',"PasswordHash","PasswordSalt"])
y = datos['ModifiedDate']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

r2 = r2_score(y_test, y_pred)
print(f"R^2 score: {r2:.4f}")

R^2 score: 0.7084


In [62]:
import pickle

file = open('model.pkl', 'wb')
pickle.dump(model, file)
file.close()

In [76]:
# Crear csv para probar modelo
rand_rows = [25,46,89,160]
testcsv = pd.DataFrame(X_test).iloc[rand_rows, :]
testcsv
testcsv.to_csv('testapi.csv', index=False)
model = pickle.load(open('model.pkl','rb'))
df_testAPI = pd.read_csv('testapi.csv')
pred = model.predict(df_testAPI)
pred

array([1.14168695e+09, 1.18860480e+09, 1.15706880e+09, 1.18592640e+09])

In [83]:
import datetime
datetime.datetime.fromtimestamp(int(pred[0]))

datetime.datetime(2006, 3, 6, 17, 15, 45)